Please do proprecess on the dataset and put in curated folder

In [1]:
# import pandas as pd

In [2]:
# individual_property = pd.read_csv("../../data/raw/individual_property.csv")

Deal with missing values, top 50 school, outlier?, parse cost ... \
Don't forget comments :)

In [3]:
# work work work

In [4]:
# Output preprocessed data
# individual_property.to_csv("../../data/curated/individual_property.csv", index=False)